<h3>Librerías, paquetes y funciones usadas en el cuaderno</h3>

In [ ]:
import cv2
import numpy as np

<h3>Función para la obtención del color del parqué y de la zona</h3>

In [1]:
# Función para manejar los clics del ratón
def get_color(event, x, y, flags, param):
    global selected_colors
    if event == cv2.EVENT_LBUTTONDOWN:
        selected_color = frame[y, x]  # Obtener el color en el punto clicado
        selected_colors.append(selected_color)
        print(f"Color seleccionado (BGR): {selected_color}")
        if len(selected_colors) == 2:  # Si ya se seleccionaron dos colores, cerrar la ventana
            cv2.destroyWindow("Selecciona un punto del parqué y un punto de la zona.")

<h3>Selección de colores del parqué y de la zona para generar una máscara</h3>

In [ ]:
# Leer el video
video_path = "./assets/videos/okc-nyk.mp4"
cap = cv2.VideoCapture(video_path)

# Variables globales para almacenar los colores seleccionados
selected_colors = []

ret, frame = cap.read()  # Leer el primer fotograma
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

# Mostrar el primer fotograma para seleccionar colores
cv2.imshow("Selecciona un punto del parqué y un punto de la zona.", frame)
cv2.setMouseCallback("Selecciona un punto del parqué y un punto de la zona.", get_color)

print("1. Haz clic en el parquet para seleccionar un color.")
print("2. Haz clic en la zona bajo la canasta para seleccionar otro color.")


# Esperar hasta que el usuario seleccione dos colores
while len(selected_colors) < 2:
    cv2.waitKey(1)

# Convertir los colores seleccionados a espacio HSV
selected_colors_hsv = [cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_BGR2HSV)[0][0] for color in selected_colors]

# Crear máscaras para cada color seleccionado
masks = []
for color_hsv in selected_colors_hsv:
    lower_bound_hsv = np.clip(color_hsv - [10, 50, 50], 0, 255)
    upper_bound_hsv = np.clip(color_hsv + [10, 50, 50], 0, 255)

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(frame_hsv, lower_bound_hsv, upper_bound_hsv)
    masks.append(mask)

# Combinar las máscaras
combined_mask = cv2.bitwise_or(masks[0], masks[1])

# Invertir la máscara combinada y aplicar para eliminar las áreas seleccionadas
mask_inv = cv2.bitwise_not(combined_mask)
masked_frame = cv2.bitwise_and(frame, frame, mask=mask_inv)

# Mostrar los resultados
cv2.imshow("Original", frame)
cv2.imshow("Parque y zona eliminadas", masked_frame)

cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()
